## 1. Setup and Data Loading

In [ ]:
# Import libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib
from pathlib import Path

# ML libraries
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, auc, roc_auc_score,
    precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize

# Add src to path
sys.path.append('../src')
from business_evaluator import BusinessEvaluator
from utils import *

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")

In [ ]:
# Load test data
data_dir = Path('../data/processed')
X_test = pd.read_csv(data_dir / 'X_test.csv')
y_test = pd.read_csv(data_dir / 'y_test.csv').values.ravel()

print(f"Test data shape: {X_test.shape}")
print(f"Target distribution:\n{pd.Series(y_test).value_counts().sort_index()}")

# Class names
class_names = {
    0: 'Offer Received',
    1: 'Offer Viewed',
    2: 'Transaction',
    3: 'Offer Completed'
}

print(f"\n✓ Loaded {len(X_test)} test samples")

In [ ]:
# Load all trained models
models_dir = Path('../models')
models = {}

# Load XGBoost Standard
try:
    models['XGBoost Standard'] = joblib.load(models_dir / 'xgboost_model.pkl')
    print("✓ Loaded XGBoost Standard")
except Exception as e:
    print(f"✗ Failed to load XGBoost Standard: {e}")

# Load XGBoost Resampled
try:
    models['XGBoost Resampled'] = joblib.load(models_dir / 'xgboost_resampled_model.pkl')
    print("✓ Loaded XGBoost Resampled")
except Exception as e:
    print(f"✗ Failed to load XGBoost Resampled: {e}")

# Load Random Forest
try:
    models['Random Forest'] = joblib.load(models_dir / 'random_forest_model.pkl')
    print("✓ Loaded Random Forest")
except Exception as e:
    print(f"✗ Failed to load Random Forest: {e}")

# Try loading DNN
try:
    from tensorflow import keras
    models['DNN Entity Embedding'] = keras.models.load_model(models_dir / 'dnn_entity_embedding.h5')
    print("✓ Loaded DNN Entity Embedding")
except Exception as e:
    print(f"✗ Failed to load DNN: {e}")

print(f"\nTotal models loaded: {len(models)}")

## 2. Generate Predictions for All Models

In [ ]:
# Generate predictions and probabilities
predictions = {}
probabilities = {}

for model_name, model in models.items():
    print(f"Generating predictions for {model_name}...")
    
    # Predictions
    y_pred = model.predict(X_test)
    predictions[model_name] = y_pred
    
    # Probabilities (if available)
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test)
        probabilities[model_name] = y_proba
        print(f"  → Predictions shape: {y_pred.shape}, Probabilities shape: {y_proba.shape}")
    else:
        print(f"  → Predictions shape: {y_pred.shape} (no probabilities)")

print("\n✓ All predictions generated")

## 3. Basic Performance Metrics

In [ ]:
# Calculate basic metrics for all models
basic_metrics = []

for model_name, y_pred in predictions.items():
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1 (Micro)': f1_score(y_test, y_pred, average='micro'),
        'F1 (Macro)': f1_score(y_test, y_pred, average='macro'),
        'F1 (Weighted)': f1_score(y_test, y_pred, average='weighted'),
        'Precision (Weighted)': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'Recall (Weighted)': recall_score(y_test, y_pred, average='weighted', zero_division=0)
    }
    basic_metrics.append(metrics)

basic_metrics_df = pd.DataFrame(basic_metrics).sort_values('F1 (Weighted)', ascending=False)
print("\n📊 BASIC PERFORMANCE METRICS")
print("="*80)
display(basic_metrics_df.round(4))

# Save to CSV
output_dir = Path('../results/metrics')
output_dir.mkdir(parents=True, exist_ok=True)
basic_metrics_df.to_csv(output_dir / 'comprehensive_basic_metrics.csv', index=False)
print("\n✓ Saved to results/metrics/comprehensive_basic_metrics.csv")

## 4. Per-Class Performance Analysis

In [ ]:
# Per-class metrics for each model
per_class_results = []

for model_name, y_pred in predictions.items():
    # Get classification report
    report = classification_report(
        y_test, y_pred, 
        target_names=[class_names[i] for i in sorted(class_names.keys())],
        output_dict=True,
        zero_division=0
    )
    
    print(f"\n{'='*80}")
    print(f"📊 {model_name} - Per-Class Performance")
    print(f"{'='*80}")
    
    for class_id, class_name in class_names.items():
        if class_name in report:
            metrics = report[class_name]
            per_class_results.append({
                'Model': model_name,
                'Class': class_name,
                'Precision': metrics['precision'],
                'Recall': metrics['recall'],
                'F1-Score': metrics['f1-score'],
                'Support': int(metrics['support'])
            })
            
            print(f"{class_name:20s}: Precision={metrics['precision']:.4f}, "
                  f"Recall={metrics['recall']:.4f}, F1={metrics['f1-score']:.4f}")

per_class_df = pd.DataFrame(per_class_results)
per_class_df.to_csv(output_dir / 'comprehensive_per_class_metrics.csv', index=False)
print("\n✓ Saved to results/metrics/comprehensive_per_class_metrics.csv")

In [ ]:
# Visualize per-class F1 scores
fig, ax = plt.subplots(figsize=(14, 6))

# Pivot for grouped bar chart
pivot_data = per_class_df.pivot(index='Class', columns='Model', values='F1-Score')
pivot_data.plot(kind='bar', ax=ax, width=0.8, edgecolor='black')

ax.set_title('Per-Class F1-Score Comparison Across Models', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('F1-Score', fontsize=12)
ax.legend(title='Model', loc='upper right')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1.0)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save
figures_dir = Path('../results/figures')
figures_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(figures_dir / 'per_class_f1_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/figures/per_class_f1_comparison.png")

## 5. ROC-AUC Analysis (Multiclass)

In [ ]:
# ROC-AUC for models with probability outputs
n_classes = len(class_names)
y_test_bin = label_binarize(y_test, classes=list(range(n_classes)))

roc_auc_results = []

for model_name, y_proba in probabilities.items():
    print(f"\n{'='*80}")
    print(f"📈 {model_name} - ROC-AUC Analysis")
    print(f"{'='*80}")
    
    # Calculate ROC curve for each class
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.ravel()
    
    for class_id in range(n_classes):
        # Calculate ROC curve
        fpr, tpr, _ = roc_curve(y_test_bin[:, class_id], y_proba[:, class_id])
        roc_auc = auc(fpr, tpr)
        
        # Store result
        roc_auc_results.append({
            'Model': model_name,
            'Class': class_names[class_id],
            'ROC-AUC': roc_auc
        })
        
        # Plot
        axes[class_id].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {roc_auc:.4f})')
        axes[class_id].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
        axes[class_id].set_xlabel('False Positive Rate', fontsize=10)
        axes[class_id].set_ylabel('True Positive Rate', fontsize=10)
        axes[class_id].set_title(f'{class_names[class_id]}', fontsize=12, fontweight='bold')
        axes[class_id].legend(loc='lower right')
        axes[class_id].grid(alpha=0.3)
        
        print(f"{class_names[class_id]:20s}: ROC-AUC = {roc_auc:.4f}")
    
    plt.suptitle(f'{model_name} - ROC Curves (One-vs-Rest)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save
    safe_name = model_name.lower().replace(' ', '_')
    plt.savefig(figures_dir / f'roc_curves_{safe_name}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate macro and micro average
    macro_auc = roc_auc_score(y_test_bin, y_proba, average='macro')
    micro_auc = roc_auc_score(y_test_bin, y_proba, average='micro')
    print(f"\nMacro-Average ROC-AUC: {macro_auc:.4f}")
    print(f"Micro-Average ROC-AUC: {micro_auc:.4f}")

# Save ROC-AUC results
roc_auc_df = pd.DataFrame(roc_auc_results)
roc_auc_df.to_csv(output_dir / 'roc_auc_scores.csv', index=False)
print("\n✓ Saved ROC-AUC results to results/metrics/roc_auc_scores.csv")

## 6. Precision-Recall Analysis (Minority Classes)

In [ ]:
# Precision-Recall curves for minority classes (1: Offer Viewed, 3: Offer Completed)
minority_classes = [1, 3]  # Offer Viewed and Offer Completed

pr_results = []

for model_name, y_proba in probabilities.items():
    print(f"\n{'='*80}")
    print(f"📊 {model_name} - Precision-Recall for Minority Classes")
    print(f"{'='*80}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for idx, class_id in enumerate(minority_classes):
        # Calculate PR curve
        precision, recall, _ = precision_recall_curve(
            y_test_bin[:, class_id], 
            y_proba[:, class_id]
        )
        avg_precision = average_precision_score(y_test_bin[:, class_id], y_proba[:, class_id])
        
        # Store result
        pr_results.append({
            'Model': model_name,
            'Class': class_names[class_id],
            'Avg Precision': avg_precision
        })
        
        # Plot
        axes[idx].plot(recall, precision, linewidth=2, label=f'AP = {avg_precision:.4f}')
        axes[idx].set_xlabel('Recall', fontsize=12)
        axes[idx].set_ylabel('Precision', fontsize=12)
        axes[idx].set_title(f'{class_names[class_id]}', fontsize=12, fontweight='bold')
        axes[idx].legend(loc='best')
        axes[idx].grid(alpha=0.3)
        axes[idx].set_xlim([0.0, 1.0])
        axes[idx].set_ylim([0.0, 1.05])
        
        print(f"{class_names[class_id]:20s}: Average Precision = {avg_precision:.4f}")
    
    plt.suptitle(f'{model_name} - Precision-Recall Curves', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save
    safe_name = model_name.lower().replace(' ', '_')
    plt.savefig(figures_dir / f'precision_recall_{safe_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Save PR results
pr_df = pd.DataFrame(pr_results)
pr_df.to_csv(output_dir / 'precision_recall_scores.csv', index=False)
print("\n✓ Saved PR results to results/metrics/precision_recall_scores.csv")

## 7. Prediction Probability Distribution Analysis

In [ ]:
# Analyze prediction confidence/probability distributions
for model_name, y_proba in probabilities.items():
    print(f"\n{'='*80}")
    print(f"📊 {model_name} - Prediction Probability Distribution")
    print(f"{'='*80}")
    
    # Get max probability for each prediction
    max_proba = np.max(y_proba, axis=1)
    predicted_class = np.argmax(y_proba, axis=1)
    
    # Statistics
    print(f"\nConfidence Statistics:")
    print(f"  Mean confidence: {max_proba.mean():.4f}")
    print(f"  Median confidence: {np.median(max_proba):.4f}")
    print(f"  Std confidence: {max_proba.std():.4f}")
    print(f"  Min confidence: {max_proba.min():.4f}")
    print(f"  Max confidence: {max_proba.max():.4f}")
    
    # Confidence by correctness
    y_pred = predictions[model_name]
    correct_mask = (y_pred == y_test)
    
    correct_conf = max_proba[correct_mask]
    incorrect_conf = max_proba[~correct_mask]
    
    print(f"\nConfidence by Correctness:")
    print(f"  Correct predictions: {correct_conf.mean():.4f} ± {correct_conf.std():.4f}")
    print(f"  Incorrect predictions: {incorrect_conf.mean():.4f} ± {incorrect_conf.std():.4f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Overall probability distribution
    axes[0].hist(max_proba, bins=50, edgecolor='black', alpha=0.7)
    axes[0].axvline(max_proba.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {max_proba.mean():.3f}')
    axes[0].set_xlabel('Maximum Probability', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Overall Confidence Distribution', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # 2. Confidence by correctness
    axes[1].hist(correct_conf, bins=30, alpha=0.7, label='Correct', color='green', edgecolor='black')
    axes[1].hist(incorrect_conf, bins=30, alpha=0.7, label='Incorrect', color='red', edgecolor='black')
    axes[1].set_xlabel('Maximum Probability', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Confidence: Correct vs Incorrect', fontsize=12, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    # 3. Box plot by predicted class
    conf_by_class = [max_proba[predicted_class == i] for i in range(n_classes)]
    axes[2].boxplot(conf_by_class, labels=[class_names[i] for i in range(n_classes)])
    axes[2].set_xlabel('Predicted Class', fontsize=12)
    axes[2].set_ylabel('Confidence', fontsize=12)
    axes[2].set_title('Confidence by Predicted Class', fontsize=12, fontweight='bold')
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(alpha=0.3)
    
    plt.suptitle(f'{model_name} - Confidence Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save
    safe_name = model_name.lower().replace(' ', '_')
    plt.savefig(figures_dir / f'confidence_analysis_{safe_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

## 8. Business Metrics & ROI Analysis

In [ ]:
# Initialize Business Evaluator
# Assumptions: $2 per offer sent, $10 revenue per completed offer
evaluator = BusinessEvaluator(
    offer_cost=2.0,
    completion_revenue=10.0,
    view_value=0.5,
    transaction_value=5.0
)

print("\n" + "="*80)
print("💰 BUSINESS METRICS & ROI ANALYSIS")
print("="*80)
print(f"\nCost Assumptions:")
print(f"  - Offer cost: ${evaluator.offer_cost}")
print(f"  - Completion revenue: ${evaluator.completion_revenue}")
print(f"  - View value: ${evaluator.view_value}")
print(f"  - Transaction value: ${evaluator.transaction_value}")

In [ ]:
# Calculate ROI for each model
roi_results = []

for model_name, y_pred in predictions.items():
    print(f"\n{'='*80}")
    print(f"💰 {model_name} - ROI Analysis")
    print(f"{'='*80}")
    
    roi_metrics = evaluator.calculate_roi(y_test, y_pred, return_details=True)
    
    print(f"\nFinancial Metrics:")
    print(f"  Total Cost: ${roi_metrics['total_cost']:,.2f}")
    print(f"  Total Revenue: ${roi_metrics['total_revenue']:,.2f}")
    print(f"  Net Profit: ${roi_metrics['net_profit']:,.2f}")
    print(f"  ROI: {roi_metrics['roi_percentage']:.2f}%")
    print(f"  Cost per Prediction: ${roi_metrics['cost_per_prediction']:.4f}")
    print(f"  Revenue per Prediction: ${roi_metrics['revenue_per_prediction']:.4f}")
    
    roi_results.append({
        'Model': model_name,
        'Total_Cost': roi_metrics['total_cost'],
        'Total_Revenue': roi_metrics['total_revenue'],
        'Net_Profit': roi_metrics['net_profit'],
        'ROI_%': roi_metrics['roi_percentage'],
        'Cost_per_Prediction': roi_metrics['cost_per_prediction'],
        'Revenue_per_Prediction': roi_metrics['revenue_per_prediction']
    })

# Create DataFrame and save
roi_df = pd.DataFrame(roi_results).sort_values('ROI_%', ascending=False)
print("\n" + "="*80)
print("📊 ROI COMPARISON")
print("="*80)
display(roi_df.round(2))

roi_df.to_csv(output_dir / 'business_roi_metrics.csv', index=False)
print("\n✓ Saved to results/metrics/business_roi_metrics.csv")

In [ ]:
# Visualize ROI comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ROI percentage
roi_df.plot(x='Model', y='ROI_%', kind='bar', ax=axes[0], color='skyblue', edgecolor='black', legend=False)
axes[0].set_title('ROI by Model', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Model', fontsize=12)
axes[0].set_ylabel('ROI (%)', fontsize=12)
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0].grid(axis='y', alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Net profit
roi_df.plot(x='Model', y='Net_Profit', kind='bar', ax=axes[1], color='lightgreen', edgecolor='black', legend=False)
axes[1].set_title('Net Profit by Model', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('Net Profit ($)', fontsize=12)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[1].grid(axis='y', alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(figures_dir / 'business_roi_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/figures/business_roi_comparison.png")

## 9. Campaign Simulation

In [ ]:
# Simulate marketing campaigns with different strategies
campaign_budget = 10000  # $10,000 budget

print(f"\n{'='*80}")
print(f"🎯 MARKETING CAMPAIGN SIMULATION (Budget: ${campaign_budget:,})")
print(f"{'='*80}")

# Compare strategies for models with probabilities
comparison_results = evaluator.compare_strategies(
    X_test,
    y_test,
    {name: models[name] for name in probabilities.keys()},
    budget=campaign_budget,
    strategies=['top_probability', 'random'],
    target_classes=[None, 3]  # None = best class, 3 = Offer Completed
)

print("\n📊 Campaign Simulation Results:")
display(comparison_results.round(2))

# Save results
comparison_results.to_csv(output_dir / 'campaign_simulation_results.csv', index=False)
print("\n✓ Saved to results/metrics/campaign_simulation_results.csv")

In [ ]:
# Visualize campaign comparison
evaluator.plot_roi_comparison(
    comparison_results,
    save_path=figures_dir / 'campaign_roi_comparison.png'
)

In [ ]:
# Generate business report
report_text = evaluator.generate_business_report(
    comparison_results,
    save_path=output_dir / 'business_evaluation_report.txt'
)

print(report_text)

## 10. Error Analysis

In [ ]:
# Focus on best model for detailed error analysis
best_model_name = basic_metrics_df.iloc[0]['Model']
best_y_pred = predictions[best_model_name]
best_y_proba = probabilities.get(best_model_name)

print(f"\n{'='*80}")
print(f"🔍 ERROR ANALYSIS: {best_model_name}")
print(f"{'='*80}")

# Identify misclassifications
incorrect_mask = (best_y_pred != y_test)
correct_mask = ~incorrect_mask

n_errors = incorrect_mask.sum()
error_rate = n_errors / len(y_test) * 100

print(f"\nTotal Errors: {n_errors:,} ({error_rate:.2f}%)")
print(f"Total Correct: {correct_mask.sum():,} ({100-error_rate:.2f}%)")

In [ ]:
# Analyze error patterns
error_data = pd.DataFrame({
    'true_class': y_test[incorrect_mask],
    'pred_class': best_y_pred[incorrect_mask],
    'max_proba': np.max(best_y_proba[incorrect_mask], axis=1) if best_y_proba is not None else np.nan
})

# Add feature data
error_features = X_test[incorrect_mask].reset_index(drop=True)
error_data = pd.concat([error_data.reset_index(drop=True), error_features], axis=1)

print("\n📊 Error Distribution by True Class:")
error_by_true = error_data['true_class'].value_counts().sort_index()
for class_id, count in error_by_true.items():
    class_total = (y_test == class_id).sum()
    error_pct = count / class_total * 100
    print(f"{class_names[class_id]:20s}: {count:,} errors ({error_pct:.2f}% of class)")

print("\n📊 Error Distribution by Predicted Class:")
error_by_pred = error_data['pred_class'].value_counts().sort_index()
for class_id, count in error_by_pred.items():
    print(f"{class_names[class_id]:20s}: {count:,} wrong predictions")

In [ ]:
# Confusion matrix for errors
error_confusion = pd.crosstab(
    error_data['true_class'].map(class_names),
    error_data['pred_class'].map(class_names),
    rownames=['True Class'],
    colnames=['Predicted Class']
)

print("\n📊 Error Confusion Matrix:")
display(error_confusion)

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(error_confusion, annot=True, fmt='d', cmap='Reds', cbar_kws={'label': 'Count'})
plt.title(f'{best_model_name} - Error Patterns', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(figures_dir / f'error_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/figures/error_confusion_matrix.png")

In [ ]:
# Feature analysis of errors vs correct predictions
print("\n📊 Feature Comparison: Errors vs Correct Predictions\n")

correct_features = X_test[correct_mask]
error_features = X_test[incorrect_mask]

feature_comparison = []

for col in X_test.columns:
    correct_mean = correct_features[col].mean()
    error_mean = error_features[col].mean()
    diff = error_mean - correct_mean
    diff_pct = (diff / correct_mean * 100) if correct_mean != 0 else 0
    
    feature_comparison.append({
        'Feature': col,
        'Correct_Mean': correct_mean,
        'Error_Mean': error_mean,
        'Difference': diff,
        'Diff_%': diff_pct
    })

feature_comp_df = pd.DataFrame(feature_comparison)
feature_comp_df = feature_comp_df.sort_values('Diff_%', key=abs, ascending=False)

display(feature_comp_df.round(4))

feature_comp_df.to_csv(output_dir / 'error_feature_analysis.csv', index=False)
print("\n✓ Saved to results/metrics/error_feature_analysis.csv")

In [ ]:
# Save top 100 error cases for manual inspection
if best_y_proba is not None:
    # Sort by confidence (most confident errors first)
    error_data_sorted = error_data.sort_values('max_proba', ascending=False).head(100)
else:
    error_data_sorted = error_data.head(100)

# Map class IDs to names
error_data_sorted['true_class_name'] = error_data_sorted['true_class'].map(class_names)
error_data_sorted['pred_class_name'] = error_data_sorted['pred_class'].map(class_names)

error_data_sorted.to_csv(output_dir / 'top_100_errors.csv', index=False)
print("\n✓ Saved top 100 errors to results/metrics/top_100_errors.csv")
print("\nSample of top errors:")
display(error_data_sorted[['true_class_name', 'pred_class_name', 'max_proba']].head(10))

## 11. Summary & Recommendations

In [ ]:
# Generate comprehensive summary
summary_lines = [
    "\n" + "="*80,
    "📊 COMPREHENSIVE MODEL TESTING SUMMARY",
    "="*80,
    "",
    "1. BEST OVERALL MODEL:",
    f"   Model: {basic_metrics_df.iloc[0]['Model']}",
    f"   Accuracy: {basic_metrics_df.iloc[0]['Accuracy']:.4f}",
    f"   F1-Score (Weighted): {basic_metrics_df.iloc[0]['F1 (Weighted)']:.4f}",
    f"   ROI: {roi_df.iloc[0]['ROI_%']:.2f}%",
    f"   Net Profit: ${roi_df.iloc[0]['Net_Profit']:,.2f}",
    "",
    "2. MINORITY CLASS PERFORMANCE:",
]

# Minority class summary
for class_id in [1, 3]:
    class_data = per_class_df[
        (per_class_df['Class'] == class_names[class_id]) & 
        (per_class_df['Model'] == best_model_name)
    ]
    if len(class_data) > 0:
        row = class_data.iloc[0]
        summary_lines.append(
            f"   {class_names[class_id]}: F1={row['F1-Score']:.4f}, "
            f"Precision={row['Precision']:.4f}, Recall={row['Recall']:.4f}"
        )

summary_lines.extend([
    "",
    "3. BEST BUSINESS STRATEGY:",
    f"   Strategy: {comparison_results.iloc[0]['strategy']}",
    f"   Target: {comparison_results.iloc[0]['target_class']}",
    f"   Expected ROI: {comparison_results.iloc[0]['roi_%']:.2f}%",
    f"   Conversion Rate: {comparison_results.iloc[0]['conversion_rate_%']:.2f}%",
    "",
    "4. KEY FINDINGS:",
    f"   - Total models tested: {len(models)}",
    f"   - Best model for accuracy: {basic_metrics_df.iloc[0]['Model']}",
    f"   - Best model for ROI: {roi_df.iloc[0]['Model']}",
    f"   - Error rate: {error_rate:.2f}%",
    f"   - Most challenging class: {error_by_true.idxmax()} (Class {error_by_true.index[error_by_true.argmax()]})",
    "",
    "5. RECOMMENDATIONS:",
])

# Add recommendations based on results
best_roi = roi_df.iloc[0]['ROI_%']
if best_roi > 100:
    summary_lines.append("   ✓ STRONGLY RECOMMENDED for production deployment")
    summary_lines.append(f"     High positive ROI ({best_roi:.2f}%) ensures profitability")
elif best_roi > 50:
    summary_lines.append("   ✓ RECOMMENDED for production deployment")
    summary_lines.append(f"     Positive ROI ({best_roi:.2f}%) with room for optimization")
elif best_roi > 0:
    summary_lines.append("   ⚠ CAUTIOUSLY RECOMMENDED")
    summary_lines.append(f"     Low ROI ({best_roi:.2f}%) - consider improving model or adjusting strategy")
else:
    summary_lines.append("   ✗ NOT RECOMMENDED for deployment")
    summary_lines.append(f"     Negative ROI ({best_roi:.2f}%) - model needs improvement")

summary_lines.extend([
    "",
    "6. NEXT STEPS:",
    "   - Review error cases in results/metrics/top_100_errors.csv",
    "   - Consider ensemble methods combining multiple models",
    "   - Implement temporal validation (train on old data, test on new)",
    "   - Run production benchmarks for deployment planning",
    "   - A/B test different strategies in real campaigns",
    "",
    "="*80,
    "✓ TESTING COMPLETE",
    "="*80
])

summary_text = "\n".join(summary_lines)
print(summary_text)

# Save summary
with open(output_dir / 'comprehensive_testing_summary.txt', 'w') as f:
    f.write(summary_text)

print("\n✓ Saved summary to results/metrics/comprehensive_testing_summary.txt")

## Conclusion

This notebook has provided comprehensive testing of all trained models including:

✅ **Enhanced Performance Metrics**: ROC-AUC, Precision-Recall curves for all classes

✅ **Business Evaluation**: ROI analysis, cost-benefit calculations, campaign simulations

✅ **Error Analysis**: Deep-dive into misclassifications and error patterns

✅ **Confidence Analysis**: Prediction probability distributions and reliability assessment

All results have been saved to:
- **Metrics**: `results/metrics/`
- **Figures**: `results/figures/`

**Next Steps**:
1. Run `scripts/benchmark_models.py` for production readiness testing
2. Review `notebooks/06_temporal_validation.ipynb` for time-based validation
3. Implement A/B testing framework for real-world validation